In [1]:
from bs4 import BeautifulSoup as soup
from requests import get
from tqdm import tqdm

main_href = "https://laptop.ru"

In [2]:
products_hrefs = []
for i in tqdm(range(1, 71)):
    r = soup(get(f"https://laptop.ru/catalog/noutbuki_planshety/notebooks/?PAGEN_2={i}",
                 headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}).content,
             "html.parser")
    items = r.find("div", class_="items productList")
    products_hrefs += [main_href + x.find("a").get("href") for x in items.find_all("div", class_="productColImage")]

100%|██████████████████████████████████████████████████████████████████████████████████| 70/70 [09:31<00:00,  8.16s/it]


In [3]:
products_hrefs = list(set(products_hrefs))
print(products_hrefs)

['https://laptop.ru/catalog/product/497571/', 'https://laptop.ru/catalog/product/527494/', 'https://laptop.ru/catalog/product/514633/', 'https://laptop.ru/catalog/product/444383/', 'https://laptop.ru/catalog/product/504203/', 'https://laptop.ru/catalog/product/375889/', 'https://laptop.ru/catalog/product/482333/', 'https://laptop.ru/catalog/product/527280/', 'https://laptop.ru/catalog/product/527683/', 'https://laptop.ru/catalog/product/441942/', 'https://laptop.ru/catalog/product/526600/', 'https://laptop.ru/catalog/product/516067/', 'https://laptop.ru/catalog/product/482140/', 'https://laptop.ru/catalog/product/455118/', 'https://laptop.ru/catalog/product/552943/', 'https://laptop.ru/catalog/product/443091/', 'https://laptop.ru/catalog/product/499074/', 'https://laptop.ru/catalog/product/373475/', 'https://laptop.ru/catalog/product/527333/', 'https://laptop.ru/catalog/product/455115/', 'https://laptop.ru/catalog/product/442729/', 'https://laptop.ru/catalog/product/370141/', 'https://

In [4]:
len(products_hrefs)

1123

In [5]:
from time import time
data = []
t = time()
pbar = tqdm(products_hrefs)
for href in pbar:
    try:
        r = soup(get(href, 
                     headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}).content,
                 "html.parser")
        now_data = {}
        name = r.find("h1", class_="changeName")
        if name is None:
            continue
        name = name.text
        price = r.find("span", class_="priceVal")
        if price is not None:
            price = price.text.replace("\xa0", "")
        stats = r.find("table", class_="stats").find_all("tr")
        now_data["name"] = name
        now_data["price"] = price
        for stat in stats:
            if stat.get("class") is None or stat.get("class")[0] != "cap":
                tds = stat.find_all("td")
                name = tds[0].text.strip("\n")
                value = tds[1].text.strip("\n")
                now_data[name] = value
        data.append(now_data)
    except:
        continue
    pbar.set_postfix({'Samples count': len(data)})
    
    if len(data) > 1050:
        break

 93%|████████████████████████████████████████████████████▎   | 1050/1123 [1:15:19<05:14,  4.30s/it, Samples count=1051]


In [7]:
import pandas as pd
df = pd.DataFrame(data)
df.head()

,name,price,Гарантия,Диагональ дисплея,Разрешение дисплея,Частота обновления,Тип матрицы экрана,Процессор,Количество ядер процессора,Серия процессора.,...,Емкость встроенной памяти eMMC,Мобильная сеть,Категория,Express Card,Компоновка памяти,Внешний монитор,Тип Wi-Fi,ПО,Материал и цвет palmrest,Модель
0,Ноутбук Lenovo ThinkPad X1 Carbon 10 21CB006QRT,240574 руб.,1 год,14 дюйм,1920х1200,60 Гц,TFT IPS,Intel Core i5,12,1240P,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Ноутбук Lenovo ThinkPad T14 21AH00BCRT,157648 руб.,NaN,14 дюйм,1920х1200,NaN,TFT IPS,Intel Core i5,10,1235U,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ноутбук ASUS ASUS Zenbook 14 90NB0W95-M00CN0,124806 руб.,NaN,14 дюйм,2880x1800,NaN,OLED,AMD Ryzen 5,6,5625U,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Ноутбук F+ FLTP-5R FLTP-5R5-16512-W,51300 руб.,1 год,15.6 дюйм,1920x1080 FHD,NaN,TFT IPS,AMD Ryzen 5,6,5600U,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Ноутбук HP ProBook 445 G9 6F1U5EA,129610 руб.,1 год,14 дюйм,1920x1080 FHD,NaN,NaN,AMD Ryzen 7,8,5825U,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df.to_csv("notebook.csv")